In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#### Nesse trabalho faremos a análise exploratória de dados e criaremos um modelo de classificação que preveja a satisfação ou não do usuário. 

#### Técnicas utilizadas:

* Entender os Dados
* Limpar os Dados
* Analisar o relacionamento entre as Variáveis
* Todos os passos já vistos para execução de um modelo de ML usando sckit-learn
* Análise do resultado da execução na base de  teste


In [ ]:
# Bibliotecas
import pandas as pd
from matplotlib import cm
from IPython.display import clear_output
import numpy as np
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import statistics as sts
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from matplotlib.lines import Line2D
import statsmodels.formula.api as sm
import cufflinks as cf
from plotly.offline import iplot
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split # Importando o train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestClassifier #biblioteca para random forest
import scikitplot as skplt # importando a bilbioteca para plotar o gráfico de Matriz de Confusão
from sklearn.metrics import accuracy_score # Vamos as métricas
from sklearn.metrics import mean_squared_log_error # Importando a metrica

## Carregando Base de Dados

In [ ]:
# Carregando Base de Dados
df_treino = pd.read_csv('/kaggle/input/airline-passenger-satisfaction/train.csv')
df_teste = pd.read_csv('/kaggle/input/airline-passenger-satisfaction/test.csv')

df_treino.shape, df_teste.shape

As duas bases se referem a pesquisa de satisfação de uma mesma empresa. Dessa forma os dados serão unificados.

In [ ]:
# Unindo as bases de dados
dados = df_treino.append(df_teste)

## Avaliação inicial da Base de Dados (Observação)

In [ ]:
# Observando a tabela
dados.head()

In [ ]:
# Verificando as dimensões da tabela.
print('Dimensões do DataFrame airline-passenger-satisfaction (df_treino): ')
print('Linhas: {}'.format(df_treino.shape[0]))
print('Colunas: {}'.format(df_treino.shape[1]))
print('Dimensões do DataFrame airline-passenger-satisfaction (df_teste): ')
print('Linhas: {}'.format(df_teste.shape[0]))
print('Colunas: {}'.format(df_teste.shape[1]))
print('Dimensões do DataFrame airline-passenger-satisfaction (dados = juntando as bases): ')
print('Linhas: {}'.format(dados.shape[0]))
print('Colunas: {}'.format(dados.shape[1]))

In [ ]:
# Informações sobre a base de dados
dados.info()

In [ ]:
# Cada coluna teria quantos valores únicos?
dados.nunique()

In [ ]:
# Verificando valores únicos - nas colunas tipo "object"
dados[['Gender','Customer Type','Type of Travel','Class','satisfaction']].nunique()

In [ ]:
# Analise estatística dos dados númericos
dados.describe().T

### Verificando e Tratando os valores nulos

In [ ]:
# verificando a existência de valores nulos (NaN)
print(dados.isnull().values.any())

In [ ]:
# verificando a existência de valores nulos (NaN)
dados.isna().sum()

In [ ]:
# Verificando o percentual de valores nulos
print('Porcentagem de valores nulos: ')
((dados.isnull().sum() / len(dados)) * 100).sort_values(ascending=False)

Considerando que observamos valores nulos na variável "Arrival Delay in Minutes" e que esses valores representam apenas 0.3% do total das informações, a opção foi excluir as informações nulas.

In [ ]:
# Esse comando exclui todas as linhas que tenha algum valor nulo. Nesse segundo caso a tabela é substituída.
dados2 = dados.dropna() 

Após o comando de exclusão das linhas que possuiam informações nulas (acima), abaixo fazemos a verificação se o comando foi efetuado corretamente.

In [ ]:
# verificando a existência de valores nulos (NaN)
print(dados2.isnull().values.any())

In [ ]:
# verificando a existência de valores nulos (NaN)
# Após o tratamento, verificamos que não temos valores nulos.
dados2.isna().sum()

### Verificando se há desbalanceamento dos valores da variável target.

Não é possível observar desbalanceamento da variável target.

In [ ]:
# Avaliando a variável target
dados2['satisfaction'].value_counts()

In [ ]:
# Balanceamento da variável target (conjunto de dados)
dados2['satisfaction'].value_counts(normalize=True)

In [ ]:
# Verificando o balanceamento da variável target
satisfaction = dados2.satisfaction.value_counts()
sns.barplot(x=satisfaction.index, y=satisfaction, palette='viridis')
title = plt.title("Balanceamento da Variável Target", fontsize=13, color="black")

## Relação entre Variáveis

Medida de associação é como são chamadas as métricas utilizadas para quantificar uma relação entre duas ou mais variáveis. Portanto, visa estabelecer o quanto dois ou mais atributos são associados entre si. Essas medidas são recursos bastante úteis para realizar uma pré seleção de atributos que devem ser incluídos quando pretende-se ajustar um modelo de probabilidade.

### Análise Bivariada

#### Avaliando a correlação entre a variável target e demais variáveis

In [ ]:
# Verificando a correlação entre a variável target e as colunas abaixo
cols = [ 'Inflight wifi service','Departure/Arrival time convenient', 'Ease of Online booking',
       'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
       'Inflight entertainment', 'On-board service', 'Leg room service',
       'Baggage handling', 'Checkin service', 'Inflight service','Cleanliness']

In [ ]:
# Verificando a correlação entre a variável target e as colunas abaixo
# Modelo de código visualizado no trabalho do colega Sormani Guimarães (adaptado)
fig, ax = plt.subplots(5, 3, figsize=(15, 25))
for variable, subplot in zip(cols, ax.flatten()):
    sns.countplot(dados2[variable],hue=dados2['satisfaction'], ax=subplot, palette='viridis')
    for label in subplot.get_xticklabels():
        label.set_rotation(15)

In [ ]:
# Verificando a correlação entre a variável target e as variáveis categóricas
# Modelo de código visualizado no trabalho do colega Sormani Guimarães (adaptado)
col2 =['Gender','Customer Type','Type of Travel','Class']

fig, ax = plt.subplots(1,4, figsize=(25, 7))
for variable, subplot in zip(col2, ax.flatten()):
    sns.countplot(dados2[variable],hue=dados2['satisfaction'], ax=subplot, palette='viridis')

Com a análise bivariada é possível observar a relação de cada uma das variáveis explicativas com a variável target. 

No caso, é possível observar:

* Quando falamos em classes, o maior número de insatisfeitos estão na classe Eco.
* A insatisfação é maior nas viagens pessoas em relação às viagens de trabalho.

### Avaliando a satisfação em relação as variáveis "Age" e 'Flight Distance'

* Em relação a idade, é possível observar que pessoas entre 40 e 60 anos estão mais satisfeitas
* A satisfação é maior nos vôos mais longos

In [ ]:
# Modelo de código visualizado no trabalho da colega Érica (adaptado)

fig, ax = plt.subplots(2,1, figsize=(10,15))
cols = ['Age', 'Flight Distance']
    
sns.histplot(dados2, x=cols[0], bins=25, palette='viridis', hue="satisfaction", ax=ax[0])

sns.histplot(dados2, x=cols[1], bins=25, palette='viridis', hue="satisfaction", ax=ax[1])

### Transformação dos Dados

Considerando que as variáveis 'Unnamed:_0' e 'id' não serão utilizadas no modelo, elas serão retiradas da base de dados.

Serão transformadas, ainda, as variáveis categóricas em int.

In [ ]:
# Remoção das colunas unnamed e id
dados2.drop(['Unnamed: 0','id'],axis=1,inplace=True)

In [ ]:
# Transformando as variáveis em binarias (neutral or dissatisfied = 0, satisfied = 1).
dados2['satisfaction'] = dados2['satisfaction'].replace({'satisfied': 1, 'neutral or dissatisfied' : 0}).astype(int)
# Conferindo após transformação
dados2['satisfaction'].value_counts()

In [ ]:
# Transformando as variáveis em binarias (Female = 0, Male = 1).
dados2['Gender'] = dados2['Gender'].replace({'Male': 1, 'Female' : 0}).astype(int)
# Conferindo após transformação
dados2['Gender'].value_counts()

In [ ]:
# Transformação dos dados (Loyal = 0, disloyal = 1).
dados2['Customer Type'] = dados2['Customer Type'].replace({'Loyal Customer': 0, 'disloyal Customer' : 1}).astype(int)
# Conferindo após transformação
dados2['Customer Type'].value_counts()

In [ ]:
# Transformação dos dados (business = 0, personal = 1).
dados2['Type of Travel'] = dados2['Type of Travel'].replace({'Business travel': 0, 'Personal Travel' : 1}).astype(int)
# Conferindo após transformação
dados2['Type of Travel'].value_counts()

In [ ]:
# Transformação dos Dados (business = 0, eco = 1; eco plus = 2).
dados2['Class'] = dados2['Class'].replace({'Business': 0, 'Eco' : 1, 'Eco Plus' : 2}).astype(int)
# Conferindo após transformação
dados2['Class'].value_counts()

In [ ]:
# Informações sobre a base de dados após a transformação
dados2.info()

In [ ]:
# Visualização dos Dados após as trasnformações
dados2.head()

#### Representação visual e nemérica do grau de correlação entre as variáveis (Mapa de Calor)

Obs 1: Valores próximos a 0 indicam baixa ou nenhuma correlação;
Obs 2: Valores próximos a próximos a 1 ou -1 refletem alta correlação (próximo a 1 = diretamente proporcional e próximo a -1 = inversamente proporcional).
Obs 3: A correlação mede a intensidade da relação, no entanto, não significa causalidade entre elas.

In [ ]:
# Criando uma nova variável - dumarização da variável target
def satisfac(string):
    if string == 'satisfied': return 1
    else: return 0

dados2['satisfaction_cont'] =dados2['satisfaction'].apply(satisfac)

In [ ]:
#Grafico correlação

plt.figure(figsize=(20,15))
sns.heatmap(dados2.corr(),annot=True,cmap='viridis')
plt.tight_layout
title = plt.title("Mapa de Calor - Gráfico de Correlação", fontsize=13, color="black")

É possível observar grande relação entre as variáveis 'Arrival Delay in Minutes' e 'Departure Delay in Minutes'.

In [ ]:
# Excluindo a coluna coluna criada (drop)
dados2.drop('satisfaction_cont',axis=1,inplace=True)

### Avaliado a base de dados e a escolha do modelo a ser utilizado:

* Com base nas informações coletadas até o momento (correlação entre as variáveis), criaremos um modelo de predição baseado em random forest.

In [ ]:
rfc = RandomForestClassifier(n_jobs=-1, n_estimators=200, random_state=90)
rfc2 = RandomForestClassifier(n_jobs=-1, n_estimators=200, random_state=90)

In [ ]:
# Separando as colunas que não são target para treinamento
feats = [c for c in dados2.columns if c not in ['satisfaction']]
feats

O próximo passo é difinir qual a parte da base que utilizaremos para treinar e para testar o modelo. Não podemos utilizar todos os registros para treino, para não perder a referência de como o modelo funciona com dados desconhecidos. 

In [ ]:
# Separando as bases de treino e teste
treino, teste = train_test_split(dados2, test_size=0.20, random_state=90)

# Separando as bases de treino e validação
treino, validacao = train_test_split(treino, test_size=0.20, random_state=90)

treino.shape, validacao.shape, teste.shape

In [ ]:
# Treinando o modelo - Treino
rfc.fit(treino[feats], treino['satisfaction'])

In [ ]:
# Treinando o modelo - Validacao
rfc2.fit(validacao[feats], validacao['satisfaction'])

In [ ]:
# Prevendo os dados de validação
preds_validacao = rfc2.predict(validacao[feats])

preds_validacao

In [ ]:
# Acurácia das previsões de validação
accuracy_score(validacao['satisfaction'], preds_validacao)

In [ ]:
# Medindo a acurácia nos dados de teste
preds_teste = rfc.predict(teste[feats])

accuracy_score(teste['satisfaction'], preds_teste)

In [ ]:
# Avaliando a importancia de cada coluna (cada variável de entrada) - Teste
pd.Series(rfc.feature_importances_, index=feats).sort_values().plot.barh()
title = plt.title("Importância de cada variável - Teste", fontsize=13, color="black")

Observando o gráfico acima, verificamos que a variável online boarding foi a que mais influenciou no modelo.

In [ ]:
# Avaliando a importancia de cada coluna (cada variável de entrada) - Validação
pd.Series(rfc2.feature_importances_, index=feats).sort_values().plot.barh()
title = plt.title("Importância de cada variável - Validação", fontsize=13, color="black")

Observando o gráfico acima, verificamos que a variável online boarding foi a que mais influenciou no modelo.

In [ ]:
# Matriz de Confusão - Dados de Validação
skplt.metrics.plot_confusion_matrix(validacao['satisfaction'], preds_validacao, cmap='viridis')

In [ ]:
# Matriz de Confusão - Dados de TESTE
skplt.metrics.plot_confusion_matrix(teste['satisfaction'], preds_teste, cmap='viridis')

O modelo utilizado obteve um bom desempenho, apresentando uma acurácia de 96,35%. Pelos gráficos apresentados, as variáveis "Online boarding" e "Inflight wifi service" tiveram maior impacto na satisfação dos passageiros.